# Green Trap Analysis

Use this notebook to explore clustering and machine learning analyses on the processed WDI features.

In [ ]:
import pandas as pd
from pathlib import Path

from src import config

raw = pd.read_csv(config.RAW_DATA_PATH) if Path(config.RAW_DATA_PATH).exists() else None
features = pd.read_csv(config.FEATURES_PATH) if Path(config.FEATURES_PATH).exists() else None

(raw.head(), features.head()) if raw is not None and features is not None else 'Add data to begin analysis.'